In [1]:
# 필요 패키지 호출
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.parse
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import time
import re
import pyautogui
import random
import psycopg2
import pandas as pd
import urllib.request
import json


### instagram 크롤링을 위한 함수 정의 및 준비

In [30]:
## 리스트 
        
CAMPING_dic = {'#캠핑':0, '#캠핑요리':0, '#캠핑용품':0, '#캠핑소품':0, '#캠핑그램':0, '#캠퍼':0, '#캠린이':0, '#감성캠핑':0, 
'#차박':0, '#글램핑':0, '#오토캠핑':0, '#카라반캠핑':0, '#카라반':0, '#노지캠핑':0, '#오지캠핑':0,
'#캠핑요리':0, '#캠핑음식':0, '#캠핑스타그램':0, '#캠핑장':0, '#캠핑카':0, '#불멍':0, '#백패커':0}

camping = list(CAMPING_dic.keys())

Camping_ls = []
for i in camping:
        Camping_ls.append(i.replace('#',''))

In [31]:
Camping_ls

['캠핑',
 '캠핑요리',
 '캠핑용품',
 '캠핑소품',
 '캠핑그램',
 '캠퍼',
 '캠린이',
 '감성캠핑',
 '차박',
 '글램핑',
 '오토캠핑',
 '카라반캠핑',
 '카라반',
 '노지캠핑',
 '오지캠핑',
 '캠핑음식',
 '캠핑스타그램',
 '캠핑장',
 '캠핑카',
 '불멍',
 '백패커']

In [2]:
# 함수 정의 : 검색어 조건에 따른 url 생성
def insta_searching(word):
    url = f"https://www.instagram.com/explore/tags/{word}"
    return url

In [3]:
# 함수정의 : 열린 페이지에서 첫번째 게시물 클릭 + sleep 메소드 통하여 시차 두기
def select_first(driver):
    first = driver.find_elements_by_css_selector('div._9AhH0')[0]
    first.click()
    time.sleep(random.randint(2,5))

In [4]:
# 첫 번째 게시물 클릭 후 다음 게시물 클릭
def move_next(driver):
    right = driver.find_element_by_css_selector("div.l8mY4.feth3")
    right.click()
    time.sleep(6)

In [5]:
def reple(): # 댓글 플러스 버튼 누르기  
    while True:
        try :
            button = driver.find_element_by_css_selector('body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.eo2As > div.EtaWk > ul > li > div > button')
            button.click()
            time.sleep(3)
        except : 
            break

In [6]:
def re_reple(): # 대댓글 버튼 누르기

    buttons = driver.find_elements_by_css_selector('li > ul > li > div > button')

    for button in buttons:
        button.send_keys(Keys.ENTER)
        time.sleep(2)

In [7]:
def reple_id():  # 인스타 댓글의 id 카운팅
    id_f = []
    ids  = driver.find_elements_by_css_selector('div.C4VMK > h3 > div > span > a')
    for id in ids:
        id_a = id.text.strip()
        id_f.append(id_a)
    return len(id_f)

In [8]:
# 함수 정의 : 본문 내용, 작성일자, 좋아요 수, 해시태그 가져오기
import re
from bs4 import BeautifulSoup



def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    

    # 본문 내용 
    try:
        content = soup.select('div.MOdxS')[0].text 
    except:
        content = ' '
    
    


    # 해시태그 
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자 
    date = soup.select('time')[0]['datetime'][:10]
    # 좋아요
    try:
        like = soup.select('section.EDfFK.ygqzn > div > div > div > a > div > span')[-1].text
    except:
        like = 0

    data = [reple_id(),content, tags, date, like]  # 댓글 수 카운팅, 본문, 해시태그, 날짜, 좋아요 수
   

    return data

In [10]:
# 크롤링 시작
"""
driver.get(url)을 통해 검색 페이지 접속하고,
target 변수에 크롤링할 게시글의 수를 바인딩
"""

# 인스타그램 로그인을 위한 계정 정보
username = pyautogui.prompt("인스타 아이디를 입력하세요.")
userpw = pyautogui.prompt("비밀번호를 입력하세요.")
time.sleep(3)

# 크롬 브라우저 열기
driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(r'https://www.instagram.com')
# loginUrl = 'https://www.instagram.com/accounts/login/'
time.sleep(random.randint(3,5))

# login
driver.find_element_by_name('username').send_keys(username)
driver.find_element_by_name('password').send_keys(userpw)
time.sleep(random.randint(1,2))
driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF').click()
time.sleep(random.randint(3,5))

# 정보 나중에 저장하기 클릭하고 넘어가기
driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF').click()
time.sleep(7)
# 설정 나중에하기 클릭하고 넘어가기
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(7)


# 게시물을 조회할 검색 키워드 입력 요청
# word = pyautogui.prompt("검색어를 입력하세요.")
word = pyautogui.prompt("검색어를 입력하세요.")
url = insta_searching(word)

# 검색 결과 페이지 열기
driver.get(url)
time.sleep(random.randint(5,8))

# 첫 번째 게시물 클릭
select_first(driver)

# 본격적으로 데이터 수집시작
# 수집할 게시물의 수 (최근 500개)
contents = []
target = int(pyautogui.prompt("크롤링할 범위를 숫자로 설정해주세요. "))



for n in range(target):
    print(n)
    
    try: # 날짜, 내용, 해시태그, 좋아요 수
        reple()
        re_reple()
        abb = get_content(driver)
        contents.append(abb)
        print(contents[n][:10])
        move_next(driver)
    except:
        print('fail')

C:\Users\User\AppData\Local\Temp/ipykernel_15500/1759250432.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('username').send_keys(username)
C:\Users\User\AppData\Local\Temp/ipykernel_15500/1759250432.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_name('password').send_keys(userpw)
C:\Users\User\AppData\Local\Temp/ipykernel_15500/1759250432.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF').click()
C:\Users\User\AppData\Local\Temp/ipykernel_15500/1759250432.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF').click()
C:\Users\User\AppData\Local\Temp/ipykernel_15500/1759250432.py:29: Depreca

0


C:\Users\User\AppData\Local\Temp/ipykernel_15500/2672829169.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  button = driver.find_element_by_css_selector('body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.eo2As > div.EtaWk > ul > li > div > button')


[0, '오랜만에 텐트와 타프 ... 진짜 진짜 진짜 힘들다 🤣강풍에 찢어지고ㅠㅠ날라가고 버라이어티 했던 하루 \U0001f979#요요캠프#솔로캠핑#미니멀캠핑#노지캠핑#오지캠핑#솔캠#텐트#힐레베르그#지프랭글러#캠핑#carcamping #camping#solocamping #solocamp #キャンプ', ['#요요캠프', '#솔로캠핑', '#미니멀캠핑', '#노지캠핑', '#오지캠핑', '#솔캠', '#텐트', '#힐레베르그', '#지프랭글러', '#캠핑', '#carcamping', '#camping', '#solocamping', '#solocamp', '#キャンプ'], '2022-04-29', '460']


C:\Users\User\AppData\Local\Temp/ipykernel_15500/3074305543.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  right = driver.find_element_by_css_selector("div.l8mY4.feth3")


1
[0, '.-✔ HARD GEAR HEXA TARP 55뜨거워지는 햇살,효과적인 자외선 차단 기능으로쾌적하게 그늘을 즐기고 싶다면?- 뛰어난 내구성의 립스탑 방식 옥스포드 300D 원단- 5m x 5.25m의 넓은 면적- UV 코팅으로 효과적인 자외선 차단- 육각 형태의 멋스러운 디자인- 가볍고 튼튼한 전용 폴대 별도 구매 가능블랙과 화이트 두 가지 색상으로 만날 수 있는시원하고 넓은 휴식공간을 즐겨보세요🖤🤍#하드기어 #타프 #헥사타프 #타프추천 #캠핑용품 #캠핑장비 #감성캠핑 #갬성캠핑 #camping #camp #camper #camplife #캠린이 #캠핑필수템 #감성캠핑용품 #캠핑용품추천 #캠핑 #캠퍼 #캠프 #캠핑스타그램 #캠핑그램', ['#하드기어', '#타프', '#헥사타프', '#타프추천', '#캠핑용품', '#캠핑장비', '#감성캠핑', '#갬성캠핑', '#camping', '#camp', '#camper', '#camplife', '#캠린이', '#캠핑필수템', '#감성캠핑용품', '#캠핑용품추천', '#캠핑', '#캠퍼', '#캠프', '#캠핑스타그램', '#캠핑그램'], '2022-04-29', '244']
2
[0, '#내맘대로..올리고 싶은 사진 올리기🏞🏕.저기에 랭글러가 서있다면....#캠핑 #백패킹 #camping #backpacking #outdoors #절벽캠핑 #キャンプ #랭글러 사고파..', ['#내맘대로..올리고', '#캠핑', '#백패킹', '#camping', '#backpacking', '#outdoors', '#절벽캠핑', '#キャンプ', '#랭글러'], '2022-04-29', '186']
3
[0, '미노캠프 모기스틱2년 만에 다시 선보입니다모기향 1개로 모기를 박멸 할수 없기에2개 1set 예전가격 그대로길어진 사이즈와 넓어진 파우치 안에 모기향까지 넣어5월 2일은 모기 & 벌레 없는 캠핑의 시작#모기향거치대#모기향#모기향케이스#모기향홀더#모기향연소기#캠핑모기향#캠핑#백패킹#차박#캠핑용품#감성용품#미노캠프#

#### 쓰지않은 원본코드

In [22]:
# 댓글 내용 추출
id_f = []
rp_f = []

ids  = driver.find_elements_by_css_selector('div.C4VMK > h3 > div > span > a')
replies = driver.find_elements_by_css_selector('div.C4VMK > div.MOdxS > span')

for id, reply in zip(ids, replies):
    id_a = id.text.strip()
    id_f.append(id_a)

    rp_a = reply.text.strip()
    rp_f.append(rp_a)

C:\Users\User\AppData\Local\Temp/ipykernel_22652/1427925327.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  ids  = driver.find_elements_by_css_selector('div.C4VMK > h3 > div > span > a')
C:\Users\User\AppData\Local\Temp/ipykernel_22652/1427925327.py:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  replies = driver.find_elements_by_css_selector('div.C4VMK > div.MOdxS > span')


In [25]:
print(id_f)

['jongdali_jeon_jong_gu', 'jang_yong_slk', 'koduckju', 'kms6274', 'lucky2w3e']


In [26]:
print(rp_f)

['<등꽃 아래서>\n\n짧은 생이라\n더\n애틋하다.....\n\n새벽2시30븐에 출발하여\n4군데 돌았다....\n\n여행하기 좋은 계절\n촬영하다 중간에 정자에서 쉬고....\n산책하다\n또\n촬영하고....\n\n이 꽃\n넘 이쁘다.\n\n#전국여행\n#경남하동\n#하동여행\n#하동물향기쉼터\n#나의기록\n#봄여행\n#사진여행\n#취미스타그램\n#꽃출사가자🌼🌻🌹🍀🌷🌸💐🌿🍂🌾🌱🌙⭐☀🐸👏👉\n#히동핫플\n#하동카페\n#1박2일\n#캠핑카\n#등꽃아래서\n#캐논이미지스토밍\n#canon\n#감성샷\n#등꽃\n#가족여행\n#캠핑\n#벤로코리아\n#벤로필터\n#부티크\n#photo  by  할리 장동건\n#미션\n#자연과  가장  자연스럽게', '멋진 작품입니다 👏👏👏', '등꽃이 질투했을듯요 😂😂😂', '👏👏👏👏👏😍😍😍😍', '등꽃 아래서 너무 아름다운 미선님...']
